# Options Pricing Methods Comparison

This notebook compares three methods for pricing European options:
- **Black-Scholes**: Closed-form analytical solution
- **Binomial Tree**: Cox-Ross-Rubinstein discrete model
- **Monte Carlo**: Stochastic simulation

We'll examine how the numerical methods (binomial and Monte Carlo) converge to the analytical Black-Scholes price.

## Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import numpy as np

from options_pricing_engine.core.option_types import Option, OptionType, ExerciseStyle
from options_pricing_engine.models import price, delta, gamma, vega, theta, rho
from options_pricing_engine.models import price_binomial, price_monte_carlo

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Define Test Option

We'll use a standard at-the-money European call option for our comparison.

In [ ]:
# At-the-money European call option
option = Option(
    spot=100.0,           # Current stock price
    strike=100.0,         # Strike price (ATM)
    rate=0.05,            # Risk-free rate (5%)
    volatility=0.20,      # Volatility (20%)
    time_to_maturity=1.0, # Time to expiration (1 year)
    option_type=OptionType.CALL,
    exercise_style=ExerciseStyle.EUROPEAN
)

print("Option Parameters:")
print(f"  Spot:       ${option.spot:.2f}")
print(f"  Strike:     ${option.strike:.2f}")
print(f"  Rate:       {option.rate:.1%}")
print(f"  Volatility: {option.volatility:.1%}")
print(f"  Maturity:   {option.time_to_maturity} year")
print(f"  Type:       {option.option_type.value.upper()}")

## Black-Scholes Price and Greeks

The Black-Scholes model provides the exact analytical price for European options. This serves as our benchmark for the numerical methods.

In [ ]:
# Compute Black-Scholes price
bs_price = price(option)

# Compute all Greeks
bs_delta = delta(option)
bs_gamma = gamma(option)
bs_vega = vega(option)
bs_theta = theta(option)
bs_rho = rho(option)

print("Black-Scholes Results")
print("=" * 40)
print(f"Price:  ${bs_price:.4f}")
print()
print("Greeks:")
print(f"  Delta: {bs_delta:.4f}")
print(f"  Gamma: {bs_gamma:.4f}")
print(f"  Vega:  {bs_vega:.4f}")
print(f"  Theta: {bs_theta:.4f} (per year)")
print(f"  Rho:   {bs_rho:.4f}")

## Binomial Tree Convergence

The binomial tree model approximates the continuous price evolution with discrete up/down movements. As the number of time steps increases, the binomial price converges to the Black-Scholes price.

### Why does it converge?

The Cox-Ross-Rubinstein model is constructed so that as the number of steps approaches infinity, the binomial distribution of log-returns converges to a normal distribution (by the Central Limit Theorem). This matches the Black-Scholes assumption of geometric Brownian motion.

In [ ]:
# Test different number of steps
steps_list = [10, 25, 50, 100, 200]
binomial_prices = []
binomial_errors = []

print("Binomial Tree Prices")
print("=" * 40)
print(f"{'Steps':<10} {'Price':<12} {'Error':<12} {'Error %'}")
print("-" * 40)

for steps in steps_list:
    bin_price = price_binomial(option, steps=steps)
    error = bin_price - bs_price
    error_pct = (error / bs_price) * 100
    
    binomial_prices.append(bin_price)
    binomial_errors.append(abs(error))
    
    print(f"{steps:<10} ${bin_price:<11.4f} {error:+.4f}      {error_pct:+.3f}%")

print("-" * 40)
print(f"{'BS Price':<10} ${bs_price:<11.4f}")

In [ ]:
# Plot binomial convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Price vs Steps
ax1.plot(steps_list, binomial_prices, 'bo-', markersize=8, linewidth=2, label='Binomial Price')
ax1.axhline(y=bs_price, color='r', linestyle='--', linewidth=2, label=f'Black-Scholes ({bs_price:.4f})')
ax1.set_xlabel('Number of Steps', fontsize=12)
ax1.set_ylabel('Option Price ($)', fontsize=12)
ax1.set_title('Binomial Tree Convergence to Black-Scholes', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Error vs Steps (log scale)
ax2.plot(steps_list, binomial_errors, 'go-', markersize=8, linewidth=2)
ax2.set_xlabel('Number of Steps', fontsize=12)
ax2.set_ylabel('Absolute Error ($)', fontsize=12)
ax2.set_title('Binomial Tree Error Reduction', fontsize=14)
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Monte Carlo Convergence

Monte Carlo pricing simulates random asset price paths and averages the discounted payoffs. The standard error of the estimate decreases as the number of paths increases.

### Why does variance decrease?

By the Law of Large Numbers, the sample mean converges to the true expected value. The standard error decreases proportionally to $1/\sqrt{n}$, where $n$ is the number of paths. This means:
- To halve the standard error, you need 4x more paths
- To reduce error by 10x, you need 100x more paths

In [ ]:
# Test different number of paths
paths_list = [5_000, 20_000, 100_000]
mc_prices = []
mc_errors = []
mc_std_errors = []

print("Monte Carlo Prices")
print("=" * 55)
print(f"{'Paths':<12} {'Price':<14} {'Std Error':<12} {'Error vs BS'}")
print("-" * 55)

for num_paths in paths_list:
    mc_price, std_error = price_monte_carlo(option, num_paths=num_paths, seed=42)
    error = mc_price - bs_price
    
    mc_prices.append(mc_price)
    mc_std_errors.append(std_error)
    mc_errors.append(error)
    
    print(f"{num_paths:<12,} ${mc_price:<13.4f} ±{std_error:<11.4f} {error:+.4f}")

print("-" * 55)
print(f"{'BS Price':<12} ${bs_price:<13.4f}")

In [ ]:
# Plot Monte Carlo convergence with error bars
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Price vs Paths with error bars
ax1.errorbar(paths_list, mc_prices, yerr=mc_std_errors, 
             fmt='bo-', markersize=8, linewidth=2, capsize=5, capthick=2,
             label='Monte Carlo Price ± 1 SE')
ax1.axhline(y=bs_price, color='r', linestyle='--', linewidth=2, 
            label=f'Black-Scholes ({bs_price:.4f})')
ax1.set_xlabel('Number of Paths', fontsize=12)
ax1.set_ylabel('Option Price ($)', fontsize=12)
ax1.set_title('Monte Carlo Convergence to Black-Scholes', fontsize=14)
ax1.legend(fontsize=10)
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)

# Standard error vs Paths
ax2.plot(paths_list, mc_std_errors, 'go-', markersize=8, linewidth=2)
ax2.set_xlabel('Number of Paths', fontsize=12)
ax2.set_ylabel('Standard Error ($)', fontsize=12)
ax2.set_title('Monte Carlo Standard Error Reduction', fontsize=14)
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Add reference line showing 1/sqrt(n) scaling
ref_paths = np.array([5000, 100000])
ref_se = mc_std_errors[0] * np.sqrt(paths_list[0] / ref_paths)
ax2.plot(ref_paths, ref_se, 'r--', alpha=0.5, label=r'$1/\sqrt{n}$ scaling')
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

## Summary

| Method | Price | Computation | Best For |
|--------|-------|-------------|----------|
| Black-Scholes | Exact | Instant | European options, Greeks |
| Binomial (200 steps) | ~0.01 error | Fast | American options |
| Monte Carlo (100k paths) | ~0.03 SE | Moderate | Path-dependent, exotics |

### Key Takeaways

1. **Binomial convergence**: Error decreases roughly as $O(1/n)$ where $n$ is the number of steps. With 200 steps, we achieve ~0.01 accuracy.

2. **Monte Carlo convergence**: Standard error decreases as $O(1/\sqrt{n})$ where $n$ is the number of paths. This slower convergence is the price we pay for the method's flexibility.

3. **Method selection**: Use Black-Scholes when possible (European options), binomial for American options, and Monte Carlo for complex path-dependent payoffs.